# ESPN March Madness Scrapers

## Initialization

In [5]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
%matplotlib inline

## Scraper 1

Scraper collects all the 364 teams involved in the NAACP as well as links that will be modified and scraped in the second scraper. Scrapes one page and one table.

In [8]:
%%time

# Set up the WebDriver
driver = webdriver.Chrome()

# URL for ESPN 
url = 'https://www.espn.com/mens-college-basketball/bpi'


# Open ESPN page
driver.get(url)

# Allow the page to load
time.sleep(3)

# Optionally, click to load more content (e.g., click the button 12 times)
for _ in range(100):
    try:
        load_button = driver.find_element(By.LINK_TEXT, "Show More")
        load_button.click()
        time.sleep(2)  # Wait for content to load
    except:
        # In case the "load-more" button is not found or clickable, pass
        pass

# Find the articles
table = driver.find_elements(By.CLASS_NAME, "Table__TBODY")

# Initialize lists to store headlines and URLs
team_name = []
urls = []

# Loop through each article and extract the headline and URL
for row in table:
    try:
        # Find individual cells or elements within the row, depending on the structure of the table
        cells = row.find_elements(By.TAG_NAME, "tr")  # Get each row in the table
        
        for cell in cells:
            # Extract the text from the cell (or row) and URL (if available)
            name = cell.text
            if name != 'just in':
                team_name.append(name)
                
                # Attempt to extract the URL if available
                link = cell.find_element(By.TAG_NAME, "a").get_attribute("href")
                urls.append(link)
    except Exception as e:
        # print(f"Error processing row: {e}")
        pass
        
# Create a DataFrame with the headlines and URLs
df = pd.DataFrame({
    'team_name': team_name[:-1],
    'URL': urls
})

# Print the resulting DataFrame
print(df)

# Close the browser
driver.quit()


Cache folder (/Users/ericmacdougall/.cache/selenium) cannot be created: Not a directory (os error 20)
Cache folder (/Users/ericmacdougall/.cache/selenium) cannot be created: Not a directory (os error 20)
Cache folder (/Users/ericmacdougall/.cache/selenium) cannot be created: Not a directory (os error 20)


                                       team_name  \
0                          Duke Blue Devils\nACC   
1                        Houston Cougars\nBig 12   
2                             Auburn Tigers\nSEC   
3                            Florida Gators\nSEC   
4                      Alabama Crimson Tide\nSEC   
..                                           ...   
359                  The Citadel Bulldogs\nSoCon   
360           Maryland Eastern Shore Hawks\nMEAC   
361                    Coppin State Eagles\nMEAC   
362       Arkansas-Pine Bluff Golden Lions\nSWAC   
363  Mississippi Valley State Delta Devils\nSWAC   

                                                   URL  
0    https://www.espn.com/mens-college-basketball/t...  
1    https://www.espn.com/mens-college-basketball/t...  
2    https://www.espn.com/mens-college-basketball/t...  
3    https://www.espn.com/mens-college-basketball/t...  
4    https://www.espn.com/mens-college-basketball/t...  
..                               

In [10]:
len(urls)

364

In [11]:
len(team_name)

365

In [12]:
'''df = pd.DataFrame({
    'Team_name': team_name[:-1],
    'URL': urls
})'''


"df = pd.DataFrame({\n    'Team_name': team_name[:-1],\n    'URL': urls\n})"

I clean up the dataframe.

In [14]:
df['Conference'] = df['team_name'].str.partition(sep='\n')[2]
df['team_name'] = df['team_name'].str.partition(sep='\n')[0]

In [15]:
df = df.sort_values(by='team_name', ascending=True).reset_index(drop=True)
df.head(20)

,team_name,URL,Conference
0,Abilene Christian Wildcats,https://www.espn.com/mens-college-basketball/t...,WAC
1,Air Force Falcons,https://www.espn.com/mens-college-basketball/t...,Mountain West
2,Akron Zips,https://www.espn.com/mens-college-basketball/t...,MAC
3,Alabama A&M Bulldogs,https://www.espn.com/mens-college-basketball/t...,SWAC
4,Alabama Crimson Tide,https://www.espn.com/mens-college-basketball/t...,SEC
5,Alabama State Hornets,https://www.espn.com/mens-college-basketball/t...,SWAC
6,Alcorn State Braves,https://www.espn.com/mens-college-basketball/t...,SWAC
7,American University Eagles,https://www.espn.com/mens-college-basketball/t...,Patriot
8,App State Mountaineers,https://www.espn.com/mens-college-basketball/t...,Sun Belt
9,Arizona State Sun Devils,https://www.espn.com/mens-college-basketball/t...,Big 12


In [16]:
df.sample(20)

,team_name,URL,Conference
209,Northern Iowa Panthers,https://www.espn.com/mens-college-basketball/t...,MVC
331,Utah State Aggies,https://www.espn.com/mens-college-basketball/t...,Mountain West
278,Southern Utah Thunderbirds,https://www.espn.com/mens-college-basketball/t...,WAC
285,Stetson Hatters,https://www.espn.com/mens-college-basketball/t...,ASUN
46,Charleston Southern Buccaneers,https://www.espn.com/mens-college-basketball/t...,Big South
43,Central Connecticut Blue Devils,https://www.espn.com/mens-college-basketball/t...,NEC
256,Sam Houston Bearkats,https://www.espn.com/mens-college-basketball/t...,CUSA
2,Akron Zips,https://www.espn.com/mens-college-basketball/t...,MAC
88,Florida International Panthers,https://www.espn.com/mens-college-basketball/t...,CUSA
291,Temple Owls,https://www.espn.com/mens-college-basketball/t...,American


https://www.espn.com/mens-college-basketball/team/_/id/248/houston-cougars

https://www.espn.com/mens-college-basketball/team/schedule/_/id/248

In [19]:
df['Schedule_link'] = df['URL'].str.replace('/_/', '/schedule' + '/_/').str.replace(r'/id/\d+/', '/schedule' + '')
# df['Schedule_link'] =  df['URL'].apply(lambda url: re.sub(r'(https://www\.espn\.com/mens-college-basketball/team)/_(/id/\d+)', r'\1/schedule/_\2/season/2024', url))

I modify the links to creaet a series of links for each team's schedule. Additional parameters were added above for the season.

In [21]:
%matplotlib inline
import random

print(df['Schedule_link'][random.randint(0, len(df))])

https://www.espn.com/mens-college-basketball/team/schedule/_/id/248/houston-cougars


The above cell tests that the data transformation was successful.

We must use Selenium because ESPN blocks the requests library.

In [24]:
pd.set_option('display.max_colwidth', 180)
df['Schedule_link'].head()

0    https://www.espn.com/mens-college-basketball/team/schedule/_/id/2000/abilene-christian-wildcats
1             https://www.espn.com/mens-college-basketball/team/schedule/_/id/2005/air-force-falcons
2                    https://www.espn.com/mens-college-basketball/team/schedule/_/id/2006/akron-zips
3           https://www.espn.com/mens-college-basketball/team/schedule/_/id/2010/alabama-am-bulldogs
4           https://www.espn.com/mens-college-basketball/team/schedule/_/id/333/alabama-crimson-tide
Name: Schedule_link, dtype: object

## Scraper 2

For reasons unknown, a large number of teams escaped our scraper on first screening. We will scrape them again and concatenate our dataframe until all 364 teams are collected as well as the information pertinent to their schedules.

In [25]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import pandas as pd
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 364 entries, 0 to 363
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   team_name      364 non-null    object
 1   URL            364 non-null    object
 2   Conference     364 non-null    object
 3   Schedule_link  364 non-null    object
dtypes: object(4)
memory usage: 11.5+ KB


In [47]:
%%time

driver = webdriver.Chrome()

# Initialize an empty dictionary to store game data
data = {}

# Loops until all team are in the data dictionary
while df['team_name'].nunique() > len(data.keys()):
    # Find teams that are not yet in the data dictionary
    remaining_teams = df[~df['team_name'].isin(data.keys())]
    
    for i, row in remaining_teams.iterrows():
        try:
            team_name = row['team_name']  
            team_schedule_link = row['Schedule_link']  

            driver.get(team_schedule_link)

            wait = WebDriverWait(driver, 10)

            # Find the table containing the schedule
            table = wait.until(EC.presence_of_element_located((By.CLASS_NAME, "Table__TBODY")))
            
            wait.until(EC.visibility_of(table))

            table = driver.find_elements(By.CLASS_NAME, "Table__TBODY")
            if not table:
                data[team_name] = 'No available data'
                continue  # Skip if the table is not found

            # Extract the text content of the table
            lines = table[0].text.strip()

            # Store the extracted lines in the data dictionary with the team name as the key
            if lines and lines != 'No available data':  # Ensure non-empty data
                data[team_name] = lines
            else:
                pass

        except Exception as e:
            pass
            
    # Print progress (this will be shown only once per loop)
    print(f"Progress: {len(data.keys())} / {df['team_name'].nunique()} teams processed")

df_2025 = pd.DataFrame(list(data.items()), columns=['team_name', 'Schedule_data'])

print(df_2025)

# Close browser
driver.quit()


Cache folder (/Users/ericmacdougall/.cache/selenium) cannot be created: Not a directory (os error 20)
Cache folder (/Users/ericmacdougall/.cache/selenium) cannot be created: Not a directory (os error 20)
Cache folder (/Users/ericmacdougall/.cache/selenium) cannot be created: Not a directory (os error 20)


Progress: 364 / 364 teams processed
                      team_name  \
0    Abilene Christian Wildcats   
1             Air Force Falcons   
2                    Akron Zips   
3          Alabama A&M Bulldogs   
4          Alabama Crimson Tide   
..                          ...   
359        Wright State Raiders   
360             Wyoming Cowboys   
361           Xavier Musketeers   
362               Yale Bulldogs   
363   Youngstown State Penguins   

                                                                                                                                                                           Schedule_data  
0    Regular Season\nDATE OPPONENT RESULT W-L (CONF) HIGH POINTS HIGH REBOUNDS HIGH ASSISTS\nMon, Nov 4\nvs\nHoward Payne\nW107-74 1-0 (0-0) Bettiol 22 Hubbard 14 Smith 5\nSat, Nov ...  
1    Regular Season\nDATE OPPONENT RESULT W-L (CONF) HIGH POINTS HIGH REBOUNDS HIGH ASSISTS\nMon, Nov 4\nvs\nNorth Alabama\nL73-57 0-1 (0-0) Taylor 30 Taylor 8 Mills 4\n

In [49]:
my_dict = {}
for key, value in data.items():
    if value and value.strip(): 
        my_dict[key] = value

print(len(my_dict))

364


In [51]:
df_2025.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 364 entries, 0 to 363
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   team_name      364 non-null    object
 1   Schedule_data  364 non-null    object
dtypes: object(2)
memory usage: 5.8+ KB


In [53]:
df_2025.isna().sum()

team_name        0
Schedule_data    0
dtype: int64

In [55]:
empty_rows = df_2025[df_2025['Schedule_data'].str.strip() == '']
print(empty_rows)

empty_or_nan_rows = df_2025[df_2025['Schedule_data'].isna() | (df_2025['Schedule_data'].str.strip() == '')]
print(empty_or_nan_rows)
                 
# Filter rows where 'Schedule_data' is NaN, an empty string, or only whitespace
empty_rows = df_2025[df_2025['Schedule_data'].isna() | df_2025['Schedule_data'].str.strip().eq('')]
print(empty_rows)

Empty DataFrame
Columns: [team_name, Schedule_data]
Index: []
Empty DataFrame
Columns: [team_name, Schedule_data]
Index: []
Empty DataFrame
Columns: [team_name, Schedule_data]
Index: []


In [57]:
empty_rows.isna().sum()

team_name        0
Schedule_data    0
dtype: int64

In [61]:
df_2025['team_name'].nunique()

364

In [63]:
df_2025.isna().sum()

team_name        0
Schedule_data    0
dtype: int64

In [65]:
df_2025.sample(2)

,team_name,Schedule_data
78,Eastern Michigan Eagles,"Regular Season\nDATE OPPONENT RESULT W-L (CONF) HIGH POINTS HIGH REBOUNDS HIGH ASSISTS\nMon, Nov 4\n@\nTexas State\nL64-44 0-1 (0-0) Henry 17 Billingsley 7 Terry 5\nFri, Nov 8\..."
122,Iona Gaels,"Regular Season\nDATE OPPONENT RESULT W-L (CONF) HIGH POINTS HIGH REBOUNDS HIGH ASSISTS\nMon, Nov 4\n@\nPrinceton\nL81-80 0-1 (0-0) Njie 19 Patterson 10 Njie 6\nFri, Nov 8\n@\nH..."


In [67]:
df_2025.loc[df_2025['team_name'] == 'Syracuse Orange']

,team_name,Schedule_data
288,Syracuse Orange,"Regular Season\nDATE OPPONENT RESULT W-L (CONF) HIGH POINTS HIGH REBOUNDS HIGH ASSISTS\nMon, Nov 4\nvs\nLe Moyne\nW86-82 1-0 (0-0) Davis 22 Davis 12 Carlos 6\nTue, Nov 12\nvs\n..."


In [69]:
data['Abilene Christian Wildcats'].split('\n')[1:]

['DATE OPPONENT RESULT W-L (CONF) HIGH POINTS HIGH REBOUNDS HIGH ASSISTS',
 'Mon, Nov 4',
 'vs',
 'Howard Payne',
 'W107-74 1-0 (0-0) Bettiol 22 Hubbard 14 Smith 5',
 'Sat, Nov 9',
 'vs',
 'Middle Tennessee',
 'L79-56 1-1 (0-0) Venzant 10 Venzant 6 Madden 2',
 'Tue, Nov 12',
 'vs',
 'McMurry',
 'W101-55 2-1 (0-0) Tanner 18 Hubbard 6 Smith 4',
 'Sat, Nov 16',
 'vs',
 'Texas State',
 'W72-60 3-1 (0-0) Madden 23 Hornecker 10 Madden 4',
 'Wed, Nov 20',
 '@',
 'Kennesaw State',
 'L84-78 3-2 (0-0) Williams 19 Williams 6 Hubbard 4',
 'Mon, Nov 25',
 'vs',
 'Southern Miss *',
 'W82-74 4-2 (0-0) Bettiol 17 Hubbard 8 Smith 5',
 'Tue, Nov 26',
 '@',
 'Montana State',
 'L85-59 4-3 (0-0) Bettiol 18 Smith 6 Williams 3',
 'Sat, Nov 30',
 '@',
 'Omaha',
 'W71-55 5-3 (0-0) Bettiol 19 Bettiol 6 Williams 7',
 'Wed, Dec 4',
 '@',
 'New Mexico State',
 'W78-70 6-3 (0-0) Madden 24 Williams 8 Williams 3',
 'Mon, Dec 9',
 '@',
 'Baylor',
 'L88-57 6-4 (0-0) Bettiol 16 Hubbard 6 Madden 4',
 'Sat, Dec 14',
 'vs'

In [71]:
df_2024 = df_2025.copy()
df_2025.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 364 entries, 0 to 363
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   team_name      364 non-null    object
 1   Schedule_data  364 non-null    object
dtypes: object(2)
memory usage: 5.8+ KB


In [73]:
df_2025.iloc[-1]

team_name                                                                                                                                                                  Youngstown State Penguins
Schedule_data    Regular Season\nDATE OPPONENT RESULT W-L (CONF) HIGH POINTS HIGH REBOUNDS HIGH ASSISTS\nMon, Nov 4\nvs\nWestminster (PA)\nCANCELED\nSat, Nov 9\n@\nChicago State\nW80-60 1-0 (0-...
Name: 363, dtype: object

In [628]:
import numpy as np

data1 = []  
df_full = pd.DataFrame(columns=['HREF', 'team_name', 'date', 'location', 'opponent', 'outcome', 'score'])

while df_2025['team_name'].nunique() > df_full['team_name'].nunique():
    for key, val in data.items():
        for j in range(1, len(val.split('\n')), 4):  # Process every 4th line since the structure repeats every 4 lines
            game_data = {}
    
            try:
                # Extract game data from the string
                date = val.strip().split('\n')[j+1]
                location = val.strip().split('\n')[j+2].strip()
                opponent = val.strip().split('\n')[j+3].strip()
                outcome = val.strip().split('\n')[j+4].strip().partition(' ')[0][0]  # 'W' or 'L'
                if outcome not in ['W', 'L']:
                    outcome = val.strip().split('\n')[j+4].strip().partition(' ')[0]
                    score = np.nan
                else:
                    score = val.strip().split('\n')[j+4].strip().partition(' ')[0][1:]

    
                if date == 'DATE OPPONENT TIME TV TICKETS':
                    break  # Break if we reach the end of the data section
    
                # Collect the game data into a dictionary
                game_data['HREF'] = df.loc[df['team_name'] == key, 'Schedule_link'].iloc[0]
                game_data['team_name'] = key
                game_data['date'] = date
                game_data['location'] = location
                game_data['opponent'] = opponent
                game_data['outcome'] = outcome
                game_data['score'] = score
    
                # Append the collected game data to the list
                data1.append(game_data)
            except IndexError:
                pass  # Skip in case of an index error

    # After processing all games, concatenate the collected data into df_full
    df_full = pd.concat([df_full, pd.DataFrame(data1)], ignore_index=True)
    data1 = []

# Print the resulting DataFrame info
print(df_full.info())

# Output the final df_full
df_full


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11761 entries, 0 to 11760
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   HREF       11761 non-null  object
 1   team_name  11761 non-null  object
 2   date       11761 non-null  object
 3   location   11761 non-null  object
 4   opponent   11761 non-null  object
 5   outcome    11761 non-null  object
 6   score      11750 non-null  object
dtypes: object(7)
memory usage: 643.3+ KB
None


,HREF,team_name,date,location,opponent,outcome,score
0,https://www.espn.com/mens-college-basketball/team/schedule/_/id/2000/abilene-christian-wildcats,Abilene Christian Wildcats,"Mon, Nov 4",vs,Howard Payne,W,107-74
1,https://www.espn.com/mens-college-basketball/team/schedule/_/id/2000/abilene-christian-wildcats,Abilene Christian Wildcats,"Sat, Nov 9",vs,Middle Tennessee,L,79-56
2,https://www.espn.com/mens-college-basketball/team/schedule/_/id/2000/abilene-christian-wildcats,Abilene Christian Wildcats,"Tue, Nov 12",vs,McMurry,W,101-55
3,https://www.espn.com/mens-college-basketball/team/schedule/_/id/2000/abilene-christian-wildcats,Abilene Christian Wildcats,"Sat, Nov 16",vs,Texas State,W,72-60
4,https://www.espn.com/mens-college-basketball/team/schedule/_/id/2000/abilene-christian-wildcats,Abilene Christian Wildcats,"Wed, Nov 20",@,Kennesaw State,L,84-78
...,...,...,...,...,...,...,...
11756,https://www.espn.com/mens-college-basketball/team/schedule/_/id/2754/youngstown-state-penguins,Youngstown State Penguins,"Sun, Feb 23",vs,Green Bay,W,81-77
11757,https://www.espn.com/mens-college-basketball/team/schedule/_/id/2754/youngstown-state-penguins,Youngstown State Penguins,"Sat, Mar 1",@,Northern Kentucky,L,88-79
11758,https://www.espn.com/mens-college-basketball/team/schedule/_/id/2754/youngstown-state-penguins,Youngstown State Penguins,"Thu, Mar 6",vs,Purdue Fort Wayne,W,72-67
11759,https://www.espn.com/mens-college-basketball/team/schedule/_/id/2754/youngstown-state-penguins,Youngstown State Penguins,"Mon, Mar 10",vs,Cleveland State *,W,56-54


In [629]:
df_full.describe()

,HREF,team_name,date,location,opponent,outcome,score
count,11761,11761,11761,11761,11761,11761,11750
unique,364,364,130,2,1378,4,1633
top,https://www.espn.com/mens-college-basketball/team/schedule/_/id/2492/pepperdine-waves,Pepperdine Waves,"Mon, Nov 4",vs,Lafayette,W,72-67
freq,36,36,315,6758,32,6133,42


In [630]:
print(*[df_full[col].value_counts().head() for col in df_full.columns], sep="\n")

HREF
https://www.espn.com/mens-college-basketball/team/schedule/_/id/2492/pepperdine-waves             36
https://www.espn.com/mens-college-basketball/team/schedule/_/id/48/delaware-blue-hens             36
https://www.espn.com/mens-college-basketball/team/schedule/_/id/2754/youngstown-state-penguins    35
https://www.espn.com/mens-college-basketball/team/schedule/_/id/295/old-dominion-monarchs         35
https://www.espn.com/mens-college-basketball/team/schedule/_/id/153/north-carolina-tar-heels      35
Name: count, dtype: int64
team_name
Pepperdine Waves             36
Delaware Blue Hens           36
Youngstown State Penguins    35
Old Dominion Monarchs        35
North Carolina Tar Heels     35
Name: count, dtype: int64
date
Mon, Nov 4     315
Sat, Feb 8     309
Sat, Jan 25    298
Sat, Jan 18    295
Sat, Feb 15    292
Name: count, dtype: int64
location
vs    6758
@     5003
Name: count, dtype: int64
opponent
Lafayette             32
Lipscomb              32
Bucknell              32
R

In [631]:
df_full[~df_full['outcome'].isin(['W', 'L'])]

,HREF,team_name,date,location,opponent,outcome,score
2452,https://www.espn.com/mens-college-basketball/team/schedule/_/id/2197/eastern-illinois-panthers,Eastern Illinois Panthers,"Sat, Dec 21",@,Little Rock,POSTPONED,NaN
3406,https://www.espn.com/mens-college-basketball/team/schedule/_/id/2261/hampton-pirates,Hampton Pirates,"Thu, Feb 20",vs,Stony Brook,POSTPONED,NaN
4657,https://www.espn.com/mens-college-basketball/team/schedule/_/id/2031/little-rock-trojans,Little Rock Trojans,"Sat, Dec 21",vs,Eastern Illinois,POSTPONED,NaN
4954,https://www.espn.com/mens-college-basketball/team/schedule/_/id/2351/loyola-marymount-lions,Loyola Marymount Lions,"Sat, Jan 11",@,Pepperdine,POSTPONED,NaN
5963,https://www.espn.com/mens-college-basketball/team/schedule/_/id/93/murray-state-racers,Murray State Racers,"Sun, Feb 16",vs,Northern Iowa,POSTPONED,NaN
6491,https://www.espn.com/mens-college-basketball/team/schedule/_/id/155/north-dakota-fighting-hawks,North Dakota Fighting Hawks,"Sat, Nov 9",@,UTSA,POSTPONED,NaN
6775,https://www.espn.com/mens-college-basketball/team/schedule/_/id/2460/northern-iowa-panthers,Northern Iowa Panthers,"Sun, Feb 16",@,Murray State,POSTPONED,NaN
7381,https://www.espn.com/mens-college-basketball/team/schedule/_/id/2492/pepperdine-waves,Pepperdine Waves,"Sat, Jan 11",vs,Loyola Marymount,POSTPONED,NaN
9296,https://www.espn.com/mens-college-basketball/team/schedule/_/id/2619/stony-brook-seawolves,Stony Brook Seawolves,"Thu, Feb 20",@,Hampton,POSTPONED,NaN
10666,https://www.espn.com/mens-college-basketball/team/schedule/_/id/2636/utsa-roadrunners,UTSA Roadrunners,"Sat, Nov 9",vs,North Dakota,POSTPONED,NaN


In [632]:
df_full['team_name'].nunique()

364

In [633]:
df_full['HREF'].nunique()

364

In [634]:
counter = 0
for key, val in data.items():
    if key not in df_2025['team_name'].values:
        print(key)
        counter += 1
print(counter)

0


In [635]:
data['Alabama A&M Bulldogs']

'Regular Season\nDATE OPPONENT RESULT W-L (CONF) HIGH POINTS HIGH REBOUNDS HIGH ASSISTS\nThu, Nov 7\nvs\nOakwood\nW121-78 1-0 (0-0) Moodie 13 Crosby 8 Abdur-Rahman 6\nMon, Nov 11\nvs\nArkansas Baptist\nW103-57 2-0 (0-0) Bryant 14 Moodie 8 Ford 6\nWed, Nov 13\nvs\nFisk\nW103-69 3-0 (0-0) Riley 17 Moodie 11 Moodie 5\nSat, Nov 16\n@\nTennessee State\nL81-71 3-1 (0-0) Moodie 21 Lee 6 Ford 6\nTue, Nov 19\n@\nGeorgia\nL93-45 3-2 (0-0) Mclean 11 Smith 5 Ford 4\nFri, Nov 22\nvs\nSouth Carolina State\nL82-70 3-3 (0-0) Bryant 16 Bryant 8 Ford 5\nSat, Nov 23\nvs\nCoastal Carolina\nW77-70 4-3 (0-0) Ford 17 Moodie 7 Ford 5\nMon, Nov 25\nvs\nIU Indianapolis\nL88-83 4-4 (0-0) Ford 20 Ford 6 Ford 7\nSat, Nov 30\nvs\nLipscomb\nL82-44 4-5 (0-0) Greer 11 Bryant 5 Downey 3\nSun, Dec 15\n@\nChattanooga\nL85-63 4-6 (0-0) Bryant 19 Bryant 4 Ford 3\nWed, Dec 18\n@\nUAB\nL96-67 4-7 (0-0) Colon 12 Smith 8 Colon 3\nSat, Dec 21\nvs\nSamford\nL97-90 4-8 (0-0) Bryant 22 Bryant 4 Abdur-Rahman 10\nSat, Dec 28\n@\nGeo

In [636]:
df_full.duplicated().sum()

0

In [637]:
df_full.isna().sum()

HREF          0
team_name     0
date          0
location      0
opponent      0
outcome       0
score        11
dtype: int64

In [638]:
df_full['outcome'] = df_full['outcome'].replace({'W': 1, 'L': 0})
df_full.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11761 entries, 0 to 11760
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   HREF       11761 non-null  object
 1   team_name  11761 non-null  object
 2   date       11761 non-null  object
 3   location   11761 non-null  object
 4   opponent   11761 non-null  object
 5   outcome    11761 non-null  object
 6   score      11750 non-null  object
dtypes: object(7)
memory usage: 643.3+ KB


In [639]:
for col in df_full.columns:
    print(df_full[col].value_counts().head())
    print()

HREF
https://www.espn.com/mens-college-basketball/team/schedule/_/id/2492/pepperdine-waves             36
https://www.espn.com/mens-college-basketball/team/schedule/_/id/48/delaware-blue-hens             36
https://www.espn.com/mens-college-basketball/team/schedule/_/id/2754/youngstown-state-penguins    35
https://www.espn.com/mens-college-basketball/team/schedule/_/id/295/old-dominion-monarchs         35
https://www.espn.com/mens-college-basketball/team/schedule/_/id/153/north-carolina-tar-heels      35
Name: count, dtype: int64

team_name
Pepperdine Waves             36
Delaware Blue Hens           36
Youngstown State Penguins    35
Old Dominion Monarchs        35
North Carolina Tar Heels     35
Name: count, dtype: int64

date
Mon, Nov 4     315
Sat, Feb 8     309
Sat, Jan 25    298
Sat, Jan 18    295
Sat, Feb 15    292
Name: count, dtype: int64

location
vs    6758
@     5003
Name: count, dtype: int64

opponent
Lafayette             32
Lipscomb              32
Bucknell              

In [640]:
df_full['location'] = df_full.apply(lambda row: 'N' if '*' in row['opponent'] else row['location'], axis=1)
df_full['opponent'] = df_full['opponent'].str.replace('*', '').str.strip()
df_full['location'].value_counts()

location
vs    5520
@     5003
N     1238
Name: count, dtype: int64

In [641]:
df_full['opponent'] = df_full['opponent'].str.replace(r'^\d+|\d+$', '', regex=True).str.strip()
df_full['team_name'] = df_full['team_name'].str.replace(r'^\d+|\d+$', '', regex=True).str.strip()

In [642]:
df_full['location'] = df_full['location'].replace({'vs': 'Home', '@': 'Away', 'N': 'Neutral'})

In [643]:
df_full.shape

(11761, 7)

In [648]:
df_full.loc[0, 'score'].split('-')[1]

'74'

In [656]:
df_full['team_score'] = np.zeros(df_full.shape[0])
df_full['opponent_score'] = np.zeros(df_full.shape[0])

for index, row in df_full.iterrows():
    if not pd.isna(row['score']):
        score1, score2 = map(int, row['score'].split('-')) # Dataset has np.nan so datatype for columns would be float
        
        if row['outcome'] == 0:
            if score1 < score2:
                df_full.loc[index, 'team_score'] = score1
                df_full.loc[index, 'opponent_score'] = score2
            else:
                df_full.loc[index, 'team_score'] = score2
                df_full.loc[index, 'opponent_score'] = score1
        elif row['outcome'] == 1:
            if score1 < score2:
                df_full.loc[index, 'opponent_score'] = score1
                df_full.loc[index, 'team_score'] = score2
            else:
                df_full.loc[index, 'team_score'] = score1
                df_full.loc[index, 'opponent_score'] = score2

In [662]:
df_full.sort_values('team_name')

,HREF,team_name,date,location,opponent,outcome,score,team_score,opponent_score
0,https://www.espn.com/mens-college-basketball/team/schedule/_/id/2000/abilene-christian-wildcats,Abilene Christian Wildcats,"Mon, Nov 4",Home,Howard Payne,1,107-74,107.0,74.0
31,https://www.espn.com/mens-college-basketball/team/schedule/_/id/2000/abilene-christian-wildcats,Abilene Christian Wildcats,"Thu, Mar 13",Neutral,Seattle U,0,69-63,63.0,69.0
30,https://www.espn.com/mens-college-basketball/team/schedule/_/id/2000/abilene-christian-wildcats,Abilene Christian Wildcats,"Sat, Mar 8",Home,Grand Canyon,1,82-81,82.0,81.0
29,https://www.espn.com/mens-college-basketball/team/schedule/_/id/2000/abilene-christian-wildcats,Abilene Christian Wildcats,"Thu, Mar 6",Home,Utah Valley,0,73-60,60.0,73.0
28,https://www.espn.com/mens-college-basketball/team/schedule/_/id/2000/abilene-christian-wildcats,Abilene Christian Wildcats,"Sat, Mar 1",Away,UT Arlington,1,70-59,70.0,59.0
...,...,...,...,...,...,...,...,...,...
11738,https://www.espn.com/mens-college-basketball/team/schedule/_/id/2754/youngstown-state-penguins,Youngstown State Penguins,"Wed, Dec 18",Away,Wright State,1,80-70,80.0,70.0
11739,https://www.espn.com/mens-college-basketball/team/schedule/_/id/2754/youngstown-state-penguins,Youngstown State Penguins,"Sat, Dec 21",Away,South Carolina Upstate,1,72-64,72.0,64.0
11740,https://www.espn.com/mens-college-basketball/team/schedule/_/id/2754/youngstown-state-penguins,Youngstown State Penguins,"Sun, Dec 29",Home,Detroit Mercy,1,73-64,73.0,64.0
11742,https://www.espn.com/mens-college-basketball/team/schedule/_/id/2754/youngstown-state-penguins,Youngstown State Penguins,"Sat, Jan 4",Away,Purdue Fort Wayne,0,90-81,81.0,90.0


In [664]:
df_full.sample(20, random_state=42)

,HREF,team_name,date,location,opponent,outcome,score,team_score,opponent_score
11055,https://www.espn.com/mens-college-basketball/team/schedule/_/id/258/virginia-cavaliers,Virginia Cavaliers,"Wed, Mar 12",Neutral,Georgia Tech,0,66-60,60.0,66.0
1825,https://www.espn.com/mens-college-basketball/team/schedule/_/id/36/colorado-state-rams,Colorado State Rams,"Sat, Jan 11",Home,UNLV,1,84-62,84.0,62.0
2087,https://www.espn.com/mens-college-basketball/team/schedule/_/id/48/delaware-blue-hens,Delaware Blue Hens,"Mon, Nov 18",Home,Bryant,0,85-84,84.0,85.0
9392,https://www.espn.com/mens-college-basketball/team/schedule/_/id/2627/tarleton-state-texans,Tarleton State Texans,"Thu, Feb 13",Home,Grand Canyon,0,64-60,60.0,64.0
4681,https://www.espn.com/mens-college-basketball/team/schedule/_/id/299/long-beach-state-beach,Long Beach State Beach,"Wed, Nov 13",Away,San Francisco,0,84-54,54.0,84.0
6421,https://www.espn.com/mens-college-basketball/team/schedule/_/id/2448/north-carolina-at-aggies,North Carolina A&T Aggies,"Fri, Mar 7",Neutral,Hofstra,0,77-55,55.0,77.0
8207,https://www.espn.com/mens-college-basketball/team/schedule/_/id/139/saint-louis-billikens,Saint Louis Billikens,"Fri, Mar 14",Neutral,Loyola Chicago,0,72-64,64.0,72.0
3274,https://www.espn.com/mens-college-basketball/team/schedule/_/id/2250/gonzaga-bulldogs,Gonzaga Bulldogs,"Thu, Feb 13",Home,San Francisco,1,88-77,88.0,77.0
5405,https://www.espn.com/mens-college-basketball/team/schedule/_/id/193/miami-oh-redhawks,Miami (OH) RedHawks,"Tue, Jan 28",Home,Eastern Michigan,1,89-80,89.0,80.0
8257,https://www.espn.com/mens-college-basketball/team/schedule/_/id/2612/saint-peters-peacocks,Saint Peter's Peacocks,"Fri, Jan 31",Away,Rider,0,67-64,64.0,67.0


Dataframe is saved as a csv in a local file.

In [666]:
df_full.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11761 entries, 0 to 11760
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   HREF            11761 non-null  object 
 1   team_name       11761 non-null  object 
 2   date            11761 non-null  object 
 3   location        11761 non-null  object 
 4   opponent        11761 non-null  object 
 5   outcome         11761 non-null  object 
 6   score           11750 non-null  object 
 7   team_score      11761 non-null  float64
 8   opponent_score  11761 non-null  float64
dtypes: float64(2), object(7)
memory usage: 827.1+ KB


In [667]:
df_full['opponent'].nunique()

701

In [673]:
df_full['team_name'].nunique()

364

In [675]:
df_full['season'] = 2024
df_full.columns = df_full.columns.str.lower()

In [681]:
df_full.to_csv('2025_NBA_full_schedule.csv')